In [7]:
# importing pandas as pd
from bs4 import BeautifulSoup

# Pdf checker
import re
import pdfplumber

import requests
from io import BytesIO

from urllib.request import urlopen


In [8]:
def total_values_pdf(date):
    # Getting url link for pdf in official page
    url = 'https://coronavirus.saude.mg.gov.br/boletim'
    u = urlopen(url)
    try:
        html = u.read().decode('utf-8')
    finally:
        u.close()

    soup = BeautifulSoup(html, "html.parser")
    url_pdf_link = soup.find('a', string=lambda value: value and value.endswith(date))["href"]

    # Request and Open PDF file
    try:
        rq = requests.get(f"https://coronavirus.saude.mg.gov.br{url_pdf_link}")
        pdf_data = pdfplumber.open(BytesIO(rq.content))
    except Exception as e:
        print(e)

    # Extracting data
    text = pdf_data.pages[0].extract_text()
    total_confirmed = re.search(r'^([0-9]{1,3}\.[0-9]{3}\.[0-9]{1,3})|^([0-9]{1,3}\.[0-9]{1,3})', text, re.MULTILINE)[0]
    total_death = re.search(r'(\b([0-9]{1,3}\.[0-9]{3}\.[0-9]{1,3})\n|\b([0-9]{1,3}\.[0-9]{1,3})\n)', text, re.MULTILINE)[0]
    # removing dots and breaklines
    total_confirmed = re.sub(r"\.|\n", "", total_confirmed)
    total_death = re.sub(r"\.|\n", "", total_death)
    return [int(total_confirmed), int(total_death)]

In [9]:
print(total_values_pdf("14/05/2021"))

[1451836, 37005]
